In [1]:
import os
import matplotlib.pyplot as plt
import cv2
import pandas as pd
import numpy as np
from tensorflow.keras import applications
import pickle

In [2]:
train_folder = '/home/will/Desktop/kaggle/Whale/train'
test_folder = '/home/will/Desktop/kaggle/Whale/test'
train_preprocess_folder = '/home/will/Desktop/kaggle/Whale/train_preprocess'
test_preprocess_folder = '/home/will/Desktop/kaggle/Whale/test_preprocess'

In [3]:
if not os.path.isdir(train_preprocess_folder):
    os.mkdir(train_preprocess_folder)

In [4]:
if not os.path.isdir(test_preprocess_folder):
    os.mkdir(test_preprocess_folder)

In [5]:
preprocess_input = applications.xception.preprocess_input

In [6]:
''' this fun used for color imgs
@delayed 
def process_img(in_path,out_path,size=(256,256)): 
    img = cv2.resize(cv2.imread(in_path,cv2.IMREAD_UNCHANGED),size) 
    if len(img.shape) == 2: 
        img = np.tile(img[:,:,np.newaxis],(1, 1, 3)) 
    else: img = img[:,:,::-1] 
        img = preprocess_input(img) 
        np.save(out_path,img.astype(np.float32))'''

' this fun used for color imgs\n@delayed \ndef process_img(in_path,out_path,size=(256,256)): \n    img = cv2.resize(cv2.imread(in_path,cv2.IMREAD_UNCHANGED),size) \n    if len(img.shape) == 2: \n        img = np.tile(img[:,:,np.newaxis],(1, 1, 3)) \n    else: img = img[:,:,::-1] \n        img = preprocess_input(img) \n        np.save(out_path,img.astype(np.float32))'

Preprocess input

In [7]:
def process_img(in_path,out_path,size=(256,256)):
    img = cv2.resize(cv2.imread(in_path,cv2.IMREAD_UNCHANGED),size)
    if len(img.shape) == 3:
        img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    img = preprocess_input(img)
    np.save(out_path,img.astype(np.float32))

In [8]:
%%time
save_images  = [process_img(train_folder+'/'+img,
                            train_preprocess_folder+'/'+img.split('.')[0])
                for img in os.listdir(train_folder)]

CPU times: user 2min 3s, sys: 4.59 s, total: 2min 7s
Wall time: 2min 9s


In [9]:
%%time
save_images  = [process_img(test_folder+'/'+img,
                            test_preprocess_folder+'/'+img.split('.')[0])
                for img in os.listdir(test_folder)]

CPU times: user 40.6 s, sys: 1.5 s, total: 42.1 s
Wall time: 43 s


Forgot to make img shape [256,256,1]

In [35]:
os.listdir(train_preprocess_folder)

['9296b4c3a.npy',
 'f3ace236f.npy',
 '5d2276433.npy',
 'cb1a49d2f.npy',
 '7baac803e.npy',
 'bd6d35a80.npy',
 '8510c4f79.npy',
 'fdcf70180.npy',
 '37099f8f0.npy',
 'a5f6796d3.npy',
 '9a9976a86.npy',
 '14befe397.npy',
 '06d0f5102.npy',
 'e2b6d6773.npy',
 '27f00ce23.npy',
 'ae553c183.npy',
 'c7b7928e3.npy',
 'a17122c64.npy',
 'c21f35670.npy',
 '100f71902.npy',
 'd8a190fad.npy',
 'aa76550af.npy',
 'a828d3524.npy',
 'eeddcd5ce.npy',
 'bf84bad36.npy',
 'b8e7f76e9.npy',
 '5b6e89935.npy',
 '939ad8b8a.npy',
 'eabe9965d.npy',
 'dd0f4dda1.npy',
 '5e4ff2c5d.npy',
 'dad3da77e.npy',
 '87d850688.npy',
 '101950e9c.npy',
 '434d8b99e.npy',
 'fb17fed50.npy',
 '4bd48b6af.npy',
 '70b8dca9b.npy',
 '39f98ac18.npy',
 '7ece2a771.npy',
 'bbb058f60.npy',
 '63d13fee1.npy',
 '5ce9c4962.npy',
 '9a02cf677.npy',
 'd8fc9220d.npy',
 '7f5cb7222.npy',
 'b55d5f17a.npy',
 '5b8796cbd.npy',
 'bc7ac6577.npy',
 '1681490a7.npy',
 '26d50408d.npy',
 'ccdb19392.npy',
 'ff28b609b.npy',
 '828dc5cfc.npy',
 '1965df4d1.npy',
 'd796f62e

Split Train/Val

In [70]:
train_df = pd.read_csv('train.csv').groupby(['Id'])\
             .apply(lambda x:[item_.split('.')[0]+'.npy' for item_ in list(x.Image)]).reset_index() 
train_df.columns = ['Id', 'Imgs'] 
train_df['length'] = [len(imgs) for imgs in train_df.Imgs.tolist()]

In [71]:
train_df['val_num'] = np.floor(train_df.length/4).astype(np.int32)

In [72]:
val_df = []
for i in range(train_df.shape[0]):
    val_num = train_df.iloc[i]['val_num']
    item_i = []
    if val_num != 0:
        item_i.append(train_df.iloc[i]['Id'])
        train_list = train_df.iloc[i]['Imgs']
        np.random.shuffle(train_list)
        item_i.append(train_list[:val_num])
        del train_list[:val_num]
        val_df.append(item_i)
val_df = pd.DataFrame(val_df,columns=['Id', 'Imgs'])

In [75]:
# testing if the split works
train_df['length2'] = [len(imgs) for imgs in train_df.Imgs.tolist()]
np.all(train_df.length == train_df.val_num + train_df.length2)

True

Single out new whale

In [88]:
with open('new_whale_train.pkl', 'wb') as f:
    pickle.dump([train_preprocess_folder+'/'+item for item in train_df.iloc[0]['Imgs']], f)

In [92]:
with open('new_whale_val.pkl', 'wb') as f:
    pickle.dump([train_preprocess_folder+'/'+item for item in val_df.iloc[0]['Imgs']], f)

In [97]:
train_df = train_df.drop([0])
val_df = val_df.drop([0])

In [20]:
train_df.Imgs = train_df.Imgs.apply(lambda x:[train_preprocess_folder+'/'+item for item in x])

In [31]:
val_df.Imgs = val_df.Imgs.apply(lambda x:[train_preprocess_folder+'/'+item for item in x])

In [43]:
train_df.reset_index(drop=True)

,Id,Imgs,length,val_num,length2
0,w_0003639,[/home/will/Desktop/kaggle/Whale/train_preproc...,1,0,1
1,w_0003c59,[/home/will/Desktop/kaggle/Whale/train_preproc...,1,0,1
2,w_0027efa,[/home/will/Desktop/kaggle/Whale/train_preproc...,10,2,8
3,w_00289b1,[/home/will/Desktop/kaggle/Whale/train_preproc...,2,0,2
4,w_002c810,[/home/will/Desktop/kaggle/Whale/train_preproc...,1,0,1
5,w_0032a46,[/home/will/Desktop/kaggle/Whale/train_preproc...,1,0,1
6,w_003bae6,[/home/will/Desktop/kaggle/Whale/train_preproc...,8,2,6
7,w_00656c0,[/home/will/Desktop/kaggle/Whale/train_preproc...,2,0,2
8,w_0066399,[/home/will/Desktop/kaggle/Whale/train_preproc...,1,0,1
9,w_007fefa,[/home/will/Desktop/kaggle/Whale/train_preproc...,17,4,13


In [44]:
val_df.reset_index(drop=True)

,Id,Imgs
0,w_0027efa,[/home/will/Desktop/kaggle/Whale/train_preproc...
1,w_003bae6,[/home/will/Desktop/kaggle/Whale/train_preproc...
2,w_007fefa,[/home/will/Desktop/kaggle/Whale/train_preproc...
3,w_00904a7,[/home/will/Desktop/kaggle/Whale/train_preproc...
4,w_00d50c9,[/home/will/Desktop/kaggle/Whale/train_preproc...
5,w_010f858,[/home/will/Desktop/kaggle/Whale/train_preproc...
6,w_012678c,[/home/will/Desktop/kaggle/Whale/train_preproc...
7,w_0135f5f,[/home/will/Desktop/kaggle/Whale/train_preproc...
8,w_0189b6d,[/home/will/Desktop/kaggle/Whale/train_preproc...
9,w_01cbbbd,[/home/will/Desktop/kaggle/Whale/train_preproc...


In [45]:
# save
train_df.reset_index(drop=True).to_pickle("train_df.pkl")
val_df.reset_index(drop=True).to_pickle("val_df.pkl")